In [16]:
import gym
from gym.spaces import Discrete, Tuple, Box
import pybullet_envs

import ray
from ray import tune
from ray.tune import function
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.registry import register_env
import time

from low_level_env import LowLevelHumanoidEnv
from hier_env import HierarchicalHumanoidEnv

import timeit

from custom_callback import RewardLogCallback

import numpy as np

In [10]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

2021-03-24 23:21:45,817	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.108',
 'raylet_ip_address': '192.168.0.108',
 'redis_address': '192.168.0.108:6379',
 'object_store_address': '/tmp/ray/session_2021-03-24_23-21-45_443948_23988/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-03-24_23-21-45_443948_23988/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-03-24_23-21-45_443948_23988',
 'metrics_export_port': 52956,
 'node_id': '7f26299ccc55259baa552cea8e84297f2cedc4e3d692baa932e07dce'}

In [11]:
def make_env_low(env_config):
    import pybullet_envs
    return LowLevelHumanoidEnv()

def make_env_hier(env_config):
    import pybullet_envs
    return HierarchicalHumanoidEnv()

ENV_LOW = 'HumanoidBulletEnv-v0-Low'
register_env(ENV_LOW, make_env_low)

ENV_HIER = "HumanoidBulletEnv-v0-Hier"
register_env(ENV_HIER, make_env_hier)

TARGET_REWARD = 2000

In [12]:
config_low = {
    "env": ENV_LOW,
    "num_workers": 0,
    "num_gpus": 1,
    "monitor": False,
    "evaluation_num_episodes": 50,
    "gamma": 0.995,
    "lambda": 0.95,
    "clip_param": 0.2,
    "kl_coeff": 1.0,
    "num_sgd_iter": 20,
    "lr": .0001,
    "sgd_minibatch_size": 8000,
    "train_batch_size": 24000,
    "model": {
        "fcnet_hiddens": [256, 128, 64],
        "fcnet_activation": "tanh",
        "free_log_std": True,
    },
    "batch_mode": "complete_episodes",
    "observation_filter": "MeanStdFilter",
}

In [19]:
def policy_mapping_fn(agent_id):
    if agent_id.startswith("low_level_"):
        return "low_level_policy"
    else:
        return "high_level_policy"

single_env_hier = HierarchicalHumanoidEnv()

highLevelPolicy = (
    None,
    single_env_hier.observation_space,
    single_env_hier.intermediate_space,
    {
        "model": {
            "fcnet_hiddens": [88, 16, 8],
            "fcnet_activation": "tanh",
            "free_log_std": True,
        },
    },
)

lowLevelPolicy = (
    None,
    Box(low=-np.inf, high=np.inf, shape=[44 + 4]),
    single_env_hier.action_space,
    {
        "model": {
            "fcnet_hiddens": [256, 128, 64],
            "fcnet_activation": "tanh",
            "free_log_std": True,
        },
    },
)

config_high = {
    "env": ENV_HIER,
    "callbacks": RewardLogCallback,
    "num_workers": 0,
    "num_envs_per_worker": 1,
    "multiagent": {
        "policies": {
            "high_level_policy": highLevelPolicy,
            "low_level_policy": lowLevelPolicy,
        },
        "policy_mapping_fn": function(policy_mapping_fn),
    },
    "log_level": "WARN",
    "num_gpus": 1,
    "monitor": True,
    "evaluation_num_episodes": 50,
    "gamma": 0.995,
    "lambda": 0.95,
    "clip_param": 0.2,
    "kl_coeff": 1.0,
    "num_sgd_iter": 20,
    "lr": 0.001,
    "sgd_minibatch_size": 8000,
    "train_batch_size": 24000,
    "batch_mode": "complete_episodes",
    "observation_filter": "MeanStdFilter",
}

2021-03-24 23:23:03,715	WARNING sample.py:402 -- DeprecationWarning: wrapping <function policy_mapping_fn at 0x7fdce80521f0> with tune.function() is no longer needed


In [20]:
# agent = PPOTrainer(config_low)
agent = PPOTrainer(config_high)

2021-03-24 23:23:04,825	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'high_level_policy/log_std:0' shape=(4,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
2021-03-24 23:23:05,343	WARNING deprecation.py:33 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'low_level_policy/log_std:0' shape=(17,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an

In [21]:
# agent.restore("/home/aditya/ray_results/HWalk/PPO_HumanoidBulletEnv-v0_881b4_00000_0_2021-02-21_17-45-29/checkpoint_710/checkpoint-710")
# agent.restore("/home/aditya/ray_results/HWalk_Low/PPO_HumanoidBulletEnv-v0-Low_92895_00000_0_2021-03-20_09-42-47/checkpoint_430/checkpoint-430")
# agent.restore("/home/aditya/ray_results/HWalk_Low/PPO_HumanoidBulletEnv-v0-Low_5ab1c_00000_0_2021-03-20_11-42-54/checkpoint_805/checkpoint-805")
# agent.restore("/home/aditya/ray_results/HWalk_Low_Mimic/PPO_HumanoidBulletEnv-v0-Low_32439_00000_0_2021-03-23_22-15-18/checkpoint_545/checkpoint-545")
agent.restore("/home/aditya/ray_results/HWalk_Hier_Mimic/PPO_HumanoidBulletEnv-v0-Hier_e2e81_00000_0_2021-03-24_22-33-22/checkpoint_335/checkpoint-335")

2021-03-24 23:23:52,259	INFO trainable.py:371 -- Restored on 192.168.0.108 from checkpoint: /home/aditya/ray_results/HWalk_Hier_Mimic/PPO_HumanoidBulletEnv-v0-Hier_e2e81_00000_0_2021-03-24_22-33-22/checkpoint_335/checkpoint-335
2021-03-24 23:23:52,259	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 335, '_timesteps_total': None, '_time_total': 2576.0143282413483, '_episodes_total': 226252}


In [25]:
agent.get_weights()['high_level_policy']['high_level_policy/fc_1/kernel'].shape

(44, 88)

In [34]:
env.close()

In [77]:
env = make_env_hier(ENV_HIER)

In [78]:
env.render()
observation = env.reset()

In [80]:
# rew = []
# for i in range(5):
#     env.render()
#     observation = env.resetFromFrame(0)
#     done = False
#     tempRew = []
#     startFrame = env.frame
#     start = timeit.default_timer()
#     while(not done):
#         action = agent.compute_action(observation)
#         observation, reward, done, info = env.step(action)
#         # tempRew.append(env.calcJointScore())
#         time.sleep(1.0/60)
#     # stop = timeit.default_timer()
#     # rew.append((startFrame, tempRew, stop-start))

rew = []
for i in range(5):
    env.render()
    observation = env.resetFromFrame(0)
    done = False
    tempRew = []
    startFrame = env.frame
    start = timeit.default_timer()
    while(not done):
        action = dict()
        if('high_level_agent' in observation):
            action['high_level_agent'] = agent.compute_action(observation['high_level_agent'], policy_id='high_level_policy')
        else:
            action[env.low_level_agent_id] = agent.compute_action(observation[env.low_level_agent_id], policy_id='low_level_policy')
        observation, reward, f_done, info = env.step(action)
        done = f_done['__all__']
        # tempRew.append(env.calcJointScore())
        time.sleep(1.0/60)
    # stop = timeit.default_timer()
    # rew.append((startFrame, tempRew, stop-start))

In [81]:
env.close()

In [82]:
ray.shutdown()

In [43]:
rew[0][2]

23.287814890005393

In [44]:
len(rew[0][1])

1000

In [45]:
1000 / 23.28

42.955326460481096

In [47]:
1000 * 15/23

652.1739130434783

In [50]:
for r in rew:
    print(r[0], len(r[1]))

235 640
125 1000
210 41
140 56
117 1000
46 1000
227 78
47 1000
169 1000
187 43


In [56]:
start = timeit.default_timer()
time.sleep(3.2873985)
stop = timeit.default_timer()
print(stop-start)

3.2890066819963977
